In [17]:
# import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Code4Lib 2017 Workshop Part 2

## Merging Data

In many real world situations, the data we'd like to analyze comes in multiple files. Pandas provides multiple different ways of merging data, including "merge()", and "concat()". To get started, we first need to load each of our datasets into a DataFrame. We'll use the same surveys data as in the last part of the workshop, as well as another dataset called "species.csv", which augments the data available for each species code.

In [18]:
survey_dataframe = pd.read_csv("data/surveys.csv")
survey2_dataframe = pd.read_csv("data/surveys2.csv")

* concatenate survey with survey2, introducing duplicates
* merge survey and species datasets

http://pandas.pydata.org/pandas-docs/stable/merging.html

## Cleaning Data

* removing duplicate entries
* remove NaN entries
* run function over each row to clean a string (or something else)

## Sorting Data

* order data by weight (asc/desc)
* group by sex
* group by species

## Transforming Data

* convert m/d/y into datetime
* convert weight to different unit, demonstrating writing a function to transform data

## Statistics

* datetime stats
* any other advanced stats

## Visualization

* bar
* box
* scatterplot
* histogram
* applying to subset of data / comparison between different subsets